#Write and Load Commands
This notebook is using write and load commands and examples. <br>
It uses two temp tables based on the Deltalake sample data.



##Setup

In [0]:

##ls "/Repos/stig0509@hotmail.com/Databricks/Certification Playground/Deltalake.ipynb"


dbutils.notebook.run(
    "/Workspace/Users/stig0509@hotmail.com/Databricks/Certification Playground/Deltalake", 
    60
)



In [0]:
%sql
use catalog playground;

DROP TABLE IF EXISTS todos.temp_source_tasks;
CREATE TABLE todos.temp_source_tasks AS -- Create or replace keeps the history
SELECT * FROM todos.tasks WHERE 1=0;

DROP TABLE IF EXISTS todos.temp_target_tasks; 
CREATE TABLE todos.temp_target_tasks CLONE todos.tasks;
TRUNCATE TABLE todos.temp_target_tasks;

describe history todos.temp_target_tasks;


##Test insert overwrite
Overwrite truncates the table and inserts the new rows from the source table.<br>
However if there is a column with always identity this has to be excluded from the select statement.

In [0]:
%sql
INSERT INTO todos.temp_source_tasks (task_id, task_name, description, assigned_to, due_date)
VALUES (
  101, 
  'Design new character skin', 
  'Create concept art and assets for the upcoming seasonal character skin release.', 
  'alex.jones', 
  '2025-11-15'
);

In [0]:
%sql
INSERT OVERWRITE todos.temp_target_tasks (
    task_name,
    description,
    assigned_to,
    due_date
)
SELECT
    task_name,
    description,
    assigned_to,
    due_date
FROM
    todos.temp_source_tasks;


In [0]:
%sql
use catalog playground;
SELECT * FROM todos.temp_target_tasks;

## Test merge

In [0]:
%sql
use catalog playground;

DROP TABLE IF EXISTS todos.temp_source_tasks;
CREATE TABLE todos.temp_source_tasks AS -- Create or replace keeps the history
SELECT * FROM todos.tasks WHERE 1=0;

DROP TABLE IF EXISTS todos.temp_target_tasks; 
CREATE TABLE todos.temp_target_tasks CLONE todos.tasks;



In [0]:
%sql

INSERT INTO todos.temp_source_tasks (task_id, task_name, description, assigned_to, due_date)
VALUES (
  101, 
  'Design new character skin My New Row', 
  'Create concept art and assets for the upcoming seasonal character skin release.', 
  'Alex', 
  '2025-11-15'
);


INSERT INTO todos.temp_source_tasks (task_id,task_name, description, assigned_to, due_date) VALUES
(3,'Write Story Script My Updated Row', 'Draft the narrative and dialogue for the game.', 'Charlie', '2025-11-15')

In [0]:
%sql
select * from todos.temp_source_tasks;

In [0]:
%sql
use catalog playground;

MERGE INTO todos.temp_target_tasks AS target
USING todos.temp_source_tasks AS source
ON target.task_id = source.task_id
WHEN MATCHED THEN
  UPDATE SET
    target.task_name = source.task_name,
    target.description = source.description,
    target.assigned_to = source.assigned_to,
    target.due_date = source.due_date
WHEN NOT MATCHED THEN
  INSERT (task_name, description, assigned_to, due_date)
  VALUES (source.task_name, source.description, source.assigned_to, source.due_date);

In [0]:
%sql
select * from todos.temp_target_tasks
order by task_id;

## Testing partition by and cte's

In [0]:
%sql
use catalog playground;
with cte as (
  SELECT *, row_number() OVER (PARTITION BY task_id ORDER BY task_id) AS rwn FROM todos.temp_target_tasks
)
select * from cte;